In [12]:
from theano.sandbox import cuda

In [13]:
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [14]:
path = 'data/statefarm/'
model_path = path + 'model/'

In [15]:
model = vgg_ft(10)

In [16]:
model.load_weights(model_path+'finetune2.h5')

In [17]:
last_conv_index = [index for index,layer in enumerate(model.layers)
                    if type(layer) == Convolution2D][-1]

In [18]:
last_conv_index

30

In [19]:
conv_model = [layer for layer in model.layers[:last_conv_index + 1]]

In [20]:
conv_model = Sequential(conv_model)

In [21]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]           
___________________________________________________________________________________________

In [22]:
#now getting the fully connected layers ( used for copying weights so that we dont have to train the later layers from scratch)
fc_model = Sequential(model.layers[last_conv_index+1:])

In [23]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_10 (MaxPooling2D)   (None, 512, 7, 7)     0           convolution2d_26[0][0]           
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 25088)         0           maxpooling2d_10[0][0]            
                                                                   maxpooling2d_10[0][0]            
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 4096)          0           flatten_2[0][0]                  
                                                                   flatten_2[1][0]                  
___________________________________________________________________________________________

In [24]:
#Getting the batches 
batches = get_batches(path+'train/', batch_size=64)
val_batches = get_batches(path+'valid/', batch_size=64)

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [25]:
#onehot encoding the labels
train_classes = batches.classes
val_classes = val_batches.classes
train_labels = onehot(train_classes)
val_labels = onehot(val_classes)

In [ ]:
train_features = conv_model.predict_generator(batches, batches.N)

In [26]:
#save_array(model_path+'conv_train_features.bc', train_features)
train_features = load_array(model_path+'conv_train_features.bc')

In [44]:
train_features.shape

(17943, 512, 14, 14)

In [ ]:
val_features = conv_model.predict_generator(val_batches, val_batches.N)

In [27]:
#save_array(model_path+'conv_val_features.bc', val_features)
val_features =  load_array(model_path+'conv_val_features.bc')

In [ ]:
train_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'val_data.bc')

In [ ]:
#ToDo next : Test model with all the denser layers ( train for say 5 epochs)
#            Test model with last "relu" dense layer removed 
#            Test model with second last relu layer removed
# Maybe : Test model with just the conv layers and a final softmax layer ( just to see possible drop in accuracy as feautre extraction { dense} layers play a big role)

In [28]:
def proc_weights(layer) : return [w/2 for w in layer.get_weights()]

In [29]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [18]:
def get_fc_nd():  
    model = Sequential([
        MaxPooling2D(input_shape=conv_model.layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    
    for l1, l2 in zip(fc_model.layers, model.layers): l2.set_weights(proc_weights(l1))
        
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [19]:
fc_no_dropout = get_fc_nd()

In [20]:
fc_no_dropout.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_6 (MaxPooling2D)    (None, 512, 7, 7)     0           maxpooling2d_input_1[0][0]       
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 25088)         0           maxpooling2d_6[0][0]             
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 4096)          102764544   flatten_2[0][0]                  
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 4096)          0           dense_5[0][0]                    
___________________________________________________________________________________________

In [21]:
fc_no_dropout.fit(train_features, train_labels, batch_size=64, nb_epoch=5, validation_data=(val_features, val_labels))

Train on 17943 samples, validate on 4481 samples
Epoch 1/5
17943/17943 [==============================] - 41s - loss: 14.6790 - acc: 0.0893 - val_loss: 14.6793 - val_acc: 0.0893
Epoch 2/5
17943/17943 [==============================] - 41s - loss: 14.6790 - acc: 0.0893 - val_loss: 14.6793 - val_acc: 0.0893
Epoch 3/5
 1088/17943 [>.............................] - ETA: 37s - loss: 14.7404 - acc: 0.0855

KeyboardInterrupt: 

In [30]:
from vgg16 import Vgg16
vgg = Vgg16()

In [31]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 3, 224, 224)   0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
zeropadding2d_28 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_27[0][0]           
___________________________________________________________________________________________

In [32]:
vgg.model.load_weights(model_path+'finetune2.h5')

In [57]:
fc_model_test = vgg.model.layers[last_conv_index+1:]


In [34]:
fc_model_test.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
fc_model_test.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_15 (MaxPooling2D)   (None, 512, 7, 7)     0           convolution2d_39[0][0]           
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 25088)         0           maxpooling2d_15[0][0]            
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 4096)          102764544   flatten_3[1][0]                  
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 4096)          0           dense_9[1][0]                    
___________________________________________________________________________________________

In [36]:
fc_model_test.fit(train_features, train_labels, batch_size=64, nb_epoch=5, validation_data=(val_features, val_labels))

Exception: Error when checking model input: expected lambda_input_3 to have shape (None, 3, 224, 224) but got array with shape (17943, 512, 14, 14)

In [40]:
conv_idx = [index for index, layer in enumerate(vgg.model.layers)
               if type(layer) == Convolution2D][-1]

In [38]:
def get_fc_nd():  
    model = Sequential([
        MaxPooling2D(input_shape=conv_model.layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(1000, activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    for l1, l2 in zip(fc_model_test.layers, model.layers): l2.set_weights(proc_weights(l1))
        
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [53]:
fc_model_test.model.layers[0]

In [56]:
vgg.model.layers

In [60]:
new_model_fc = Sequential(fc_model_test)

In [63]:
new_model_fc.add(Dense(10, activation='softmax'))

In [65]:
new_model_fc.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [68]:
new_model_fc.layers.pop()

In [69]:
new_model_fc.fit(train_features, train_labels, batch_size=64, nb_epoch=5, validation_data=(val_features, val_labels))

Exception: Error when checking model input: expected lambda_input_3 to have shape (None, 3, 224, 224) but got array with shape (17943, 512, 14, 14)